In [1]:
import pandas as pd
import numpy as np
import nltk
import gensim 
from gensim.models.phrases import Phrases, Phraser
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import Counter 


In [2]:
data = pd.read_csv("data")



In [3]:
artist_gender = pd.read_excel("Artist_Gender_Complete_Excel.xlsx")

In [4]:
artist_gender_dictionary = {}
for x,y in zip(list(artist_gender.Name), list(artist_gender.Gender)):
    artist_gender_dictionary[x] = y

> Finding songs by male artists, female artists and mixed singing artists

In [5]:
male_songs = [] # [song singer name , lyrics of the song]
female_songs = []
mixed_songs = []

unknownSingers = {}
singerName = ''
# mySingerDictionary = {}
singers = list(data.songSinger)
song_index = 0
for singer in singers:
    
    # song_index shows which song is being processed
    lyrics = data.songLyrics[song_index]
    sumGender = 0
    try:
        # for this song (with song_index), find all the singers and their gender
        allSingers = []
        for individualSinger in (singer.split(",")):
            singerName = ''
            s = individualSinger.split(" ")
            if(s[0] == ''):
                length = len(s)
                if(len(s)== 3):
                    singerName = s[1] + ' '+ s[2]
                else:
                    singerName = s[1]
            else:
                l = len(s)
                for i in range(l):
                    if i == l-1:
                        singerName = singerName + s[i]
                    else:
                        singerName = singerName+s[i]+' '
                if(artist_gender_dictionary.get(singerName) == None):
                    unknownSingers[singerName] = True
                    
            allSingers.append(singerName)
            if(artist_gender_dictionary.get(singerName)!=None):
                gender = artist_gender_dictionary[singerName]
                if(gender == 'f'):
                    sumGender += 1
                else:
                    sumGender += 100
        #outside the for loop
#         print(sumGender)

        # if gender is all male, add the singers, lyrics to male_singers
        # else if gender is all female, then add singers, lyrics to female singers
        # else add to mixedSingers
        if(sumGender%100==0):
            # the song is all male
            male_songs.append([allSingers,lyrics])
        elif sumGender<100:
            # the song is all female
            female_songs.append([allSingers,lyrics])
        else:
            # song is mixed singer based
            mixed_songs.append([allSingers,lyrics])
            
    except:
        print("error at individual singer "+individualSinger)
        
    song_index+=1

error at individual singer Atif Aslam
error at individual singer  Swati Sharrma
error at individual singer Salim Merchant
error at individual singer Neeraj Shridhar
error at individual singer Megha Sriram
error at individual singer Keka Ghoshal
error at individual singer Shilpa Rao
error at individual singer Shaan


In [6]:
# for x in female_songs[:4]:
#     print(x[1])
#     print()
#     print()  
    
female_songs

[[['Samira Koppikar'],
  'Morey.. lahu ka rang kara\r\nMorey lahu ka rang kara\r\nMorey.. lahu ka rang kara\r\nMorey lahu ka rang kara\r\n\r\nKaare badarwa manwa mein ghoome\r\nKaare bhujanga sapno mein jhoome\r\nGhole.. zehar ki ye dhaara\r\nMorey lahu ka rang kara\r\nMorey.. lahu ka rang kara\r\nLahu ka rang kaara...\r\n\r\nKatit kaadank kapat kalank\r\nMaathe mandha koi shraap hai jeevan\r\nRaja ho ya rank\r\nKabila ya jhund\r\nHar praan ka hi lakshya hai maran\r\n\r\nJeetna hai jo ye rann\r\nKar bairi ka daman\r\nWarna mile na chhutkaara\r\n\r\nMorey.. lahu ka rang kara\r\nMorey lahu ka rang kara\r\nMorey.. lahu ka rang kara\r\nMorey lahu ka rang kaara...\r\nLahu ka... kaara...\r\nAa...'],
 [['Shreya Ghoshal'],
  'Badnaam shehar, badnaam gully\r\nIsme har raat hai dhoop khili\r\nIsme jo dhali main khoob chali\r\n\r\nRed red najariya...\r\nArey red red najariya\r\nBada jaalim kaptaan re\r\nRed red najariya\r\nBada jalim kaptaan re\r\nGhaav deve karajwa mein\r\nPar deve na dhyaan re\

In [37]:
import re

# preprocessing function for any song
def preprocessSong(song):
    dataset = ''
    listOfWords = re.split(r'[;,\s...\n()\'!?.]\s*',song) # gets me a list of words
    for word in listOfWords:
#         if word ==  '' || word=='(' || word==')' || word =='\'':
#             pass
#         else:
        word = word.lower()
        if word == 'x2':
            continue
        if word == 'x4':
            continue
        dataset+=' '+word
    return dataset

# function that takes female_songs, male_songs and mixed_songs, returns preprocessed_songs to feed into word2vec
def preprocessAllSongs(song2dMatrix):
    preprocessedSongLyrics = []
    
    for songPair in song2dMatrix:
        artist = songPair[0]
        lyrics = songPair[1]
        preprocessedLyrics = preprocessSong(lyrics)
        
        songsToWords = []
        for word in preprocessedLyrics.split(" "):
            if word!=' ' and word!='':
                songsToWords.append(word)
                
        preprocessedSongLyrics.append(songsToWords)
    return preprocessedSongLyrics
        
female_song_lyrics_list_of_words = (preprocessAllSongs(female_songs))
mixed_song_lyrics_list_of_words = (preprocessAllSongs(mixed_songs))

male_songs_lyrics_list_of_words = preprocessAllSongs(male_songs)
def learnWordEmbeddings(doc1):
    w2v_model1 = gensim.models.Word2Vec(doc1,size=300,window=24,min_count=2,workers=10,iter=10)
    return w2v_model1

w2vModelFemaleSingers = learnWordEmbeddings(female_song_lyrics_list_of_words)
w2vModelMale = learnWordEmbeddings(male_songs_lyrics_list_of_words)
w2vModelMixed = learnWordEmbeddings(mixed_song_lyrics_list_of_words)
def predictWordSimilarityUsingPronouns(model):
    pass

def compareResults():
    pass


In [50]:
lyrics = list(data.songLyrics)
myProcessedSong = []
for song in lyrics:
    mysong = preprocessSong(song)
    myProcessedSong.append(song)
    
data2 = data.copy()
data2["processed_lyrics"] = myProcessedSong
data2
    
    
data2.to_csv("processedSongs.csv", header = True, columns = data2.columns)

In [43]:
w2vModelMale.wv.most_similar(positive=["lamborghini"],topn=30)

[('chitti', 0.9422823786735535),
 ('fendo', 0.9342678785324097),
 ('kehar', 0.928987443447113),
 ('louis', 0.9220588207244873),
 ('jimmy', 0.9218820333480835),
 ('paid', 0.9210412502288818),
 ('6', 0.9178674221038818),
 ('pave', 0.9153310060501099),
 ('madhuri', 0.9124892950057983),
 ('seetiyan', 0.9083945751190186),
 ('vajjda', 0.9059038758277893),
 ('takkan', 0.9058851003646851),
 ('crew…', 0.9047439694404602),
 ('patola', 0.9044042825698853),
 ('vuitton', 0.9039191603660583),
 ('pahve', 0.8989903330802917),
 ('pair', 0.8970626592636108),
 ('bajaun', 0.8959270715713501),
 ('ainak', 0.8943179845809937),
 ('ankhan', 0.8911491632461548),
 ('kawwali', 0.8911310434341431),
 ('records', 0.8910359144210815),
 ('pabh', 0.8907555341720581),
 ('geetan', 0.8893426656723022),
 ('critical', 0.8881959915161133),
 ('suit', 0.8873914480209351),
 ('paule', 0.8867694735527039),
 ('nibhaaiye', 0.8856948614120483),
 ('gharan', 0.8849499821662903),
 ('chakdi', 0.8830016851425171)]

In [35]:
w2vModelMale.wv.most_similar(positive=["dabang"],topn=30)

KeyError: "word 'dabang' not in vocabulary"

In [36]:
w2vModelFemaleSingers.wv.most_similar(positive=["dabang"], topn=30)

KeyError: "word 'dabang' not in vocabulary"

In [11]:
# now lets just apply the word 2 vector model and apply lda on it




In [12]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [13]:
# Create a corpus from a list of texts
common_dictionary = Dictionary(common_texts)

In [14]:
# for i in range(10):
#     print(common_dictionary[i])
    
for text in common_texts:
    print(text)


['human', 'interface', 'computer']
['survey', 'user', 'computer', 'system', 'response', 'time']
['eps', 'user', 'interface', 'system']
['system', 'human', 'system', 'eps']
['user', 'response', 'time']
['trees']
['graph', 'trees']
['graph', 'minors', 'trees']
['graph', 'minors', 'survey']


In [15]:
common_corpus = [common_dictionary.doc2bow(text) for text in common_texts]

In [16]:
common_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [ ]:
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=10)